In [5]:
%cd '/content/drive/MyDrive/Colab Notebooks/stepik/m4. inference/w4. tasks'

/content/drive/MyDrive/Colab Notebooks/stepik/m4. inference/w4. tasks


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pandas as pd
import numpy as np
import itertools
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.proportion import proportion_confint

Загрузим данные.

In [8]:
df = pd.read_csv('credit_card_default_analysis.csv')

def centralize(df, flag):
    m = df.loc[df['default'] == flag, 'LIMIT_BAL'].median()
    df.loc[df['default'] == flag, 'LIMIT_BAL'] = df.loc[df['default'] == flag, 'LIMIT_BAL'] - m

centralize(df, 0)
centralize(df, 1)

sample1 = df[df['default'] == 0]['LIMIT_BAL'].values
sample2 = df[df['default'] == 1]['LIMIT_BAL'].values

In [9]:
def get_random_combinations(n1, n2, max_combinations=None):
    if max_combinations:
        index = list(range(n1 + n2))
        indices = set([tuple(index)])
        for i in range(max_combinations - 1):
            np.random.shuffle(index)
            indices.add(tuple(index))
        return [(index[:n1], index[n1:]) for index in indices]
    n = n1 + n2
    return [(list(index), list(filter(lambda i: i not in index, range(n))))
            for index in itertools.combinations(range(n), n1)]


def permutation_zero_dist_ind(sample1, sample2, stat_calculator, max_combinations=None):
    joined_sample = np.hstack((sample1, sample2))
    indices = get_random_combinations(len(sample1), len(sample2), max_combinations)
    distr = [stat_calculator(joined_sample[list(i[0])], joined_sample[list(i[1])]) 
    for i in indices]
    return distr


def get_cdf(sample, left, right, npoints):
    n = len(sample)
    xpoints = np.linspace(left, right, npoints, endpoint=True)
    return np.array([np.sum(sample <= x) / n for x in xpoints])


def calc_ks_stat(sample1, sample2, npoints=20):
    left = min(np.min(sample1), np.min(sample2))
    right = max(np.max(sample1), np.max(sample2))
    cdf1 = get_cdf(sample1, left, right, npoints)
    cdf2 = get_cdf(sample2, left, right, npoints)
    return np.max(np.abs(cdf1 - cdf2))


print(calc_ks_stat(sample1, sample2))
print(sorted(permutation_zero_dist_ind(sample1, sample2, calc_ks_stat, 
                                       max_combinations=100), reverse=True))


0.24944358842663927
[0.24944358842663927, 0.02043328432515301, 0.016184463942455762, 0.014947804585221491, 0.014443017232118271, 0.013882813174810604, 0.013689319095884223, 0.013420130133281805, 0.013234685408038738, 0.013188014636201673, 0.012994520557275346, 0.012847697250185974, 0.012646153817576278, 0.011919544852391883, 0.011851899322399218, 0.011833556083717056, 0.011686732776627684, 0.011106250539848483, 0.01084611710013983, 0.010690934848840872, 0.010611757071744221, 0.010459128942287066, 0.010418262992817784, 0.010186147495737652, 0.01013878014521652, 0.010092109373379565, 0.010031971413649143, 0.009945286066290082, 0.009916958533135345, 0.00989791871576895, 0.009878646705507976, 0.00970512121552669, 0.009605665258958562, 0.009511627136600254, 0.009511627136600254, 0.009510930557916186, 0.009257995097943617, 0.009171309750584555, 0.009123942400063395, 0.00887031036140673, 0.008676816282480293, 0.008630145510643339, 0.008590827513805466, 0.008562499980650617, 0.0084840187822380